Импорт необходимых библиотек

In [2]:
import requests
import time
import pandas as pd
import json

Поиск производится по ИНН заказчика -- 7704206201, сортировка -- от новейших записей к старейшим. Кроме ИНН заказчика, никаких других критериев для фильтрации результатов нет -- выгружается информация по всем контрактам. 

In [3]:
INN = 7704206201

# Тестовый запрос

Проведём тестовый запрос, чтобы из него убедиться или узнать: 
- что запрос сформирован корректно и сервер доступен; 
- общее количество контрактов, где заказчиком выступает организация с указанным ИНН (искомое число лежит в ключе total); 
- количество результатов на одной странице (искомое число лежит в ключе perpage и, по идее, если страниц несколько, должно быть 50);
- структуру данных внутри каждой записи. 

In [4]:
url = f'http://openapi.clearspending.ru/restapi/v3/contracts/select/?customerinn={INN}&sort=-signDate'
resp = requests.get(url)

print('Status code:', resp.status_code)
print('Всё ок:', resp.status_code == 200)

Status code: 200
Всё ок: True


## Общая структура данных и расчёты для парсинга

- ключ `contracts`, внутри которого лежат ключи `total`, `page`, `data` (структура описана в другом разделе блокнота), `perpage`;
- расчёт количества страниц, с которых необходимо собрать данные в процессе парсинга. 

`resp.status_code == 200` означает, что сервер доступен, запрос сформирован корректно -- какие-то данные были получены. Чтобы посмотреть эти данные, результаты запроса нужно преобразовать в json. 

In [5]:
resp_data = resp.json()
resp_data.keys()

dict_keys(['contracts'])

Внутри json лежит словарь со сложной вложенной структурой. На верхнем уровне существует только один ключ -- `contracts`, внутри которого лежит вся остальная информация. Поэтому можно перезаписать `resp_data` -- сохранить туда содержимое, хранящееся под ключом `contracts`.

In [6]:
resp_data = resp_data['contracts']
resp_data.keys()

dict_keys(['total', 'data', 'page', 'perpage'])

Среди текущих ключей: 
* `total` -- число, общее количество контрактов, в которых заказчиком выступает организация с указанным ИНН;
* `data` -- список словарей, где каждый словарь -- информация об одном контракте;
* `page` -- номер страницы;
* `perpage` -- количество контрактов, выдаваемых на одной странице. 

Зная общее количество контрактов (`total`) и количество контрактов, выдаваемых на одной странице (`perpage`), можем посчитать количество страниц, по которым нужно будет пройти при парсинге. 

In [7]:
print('Всего контрактов:', resp_data['total'])
print('Количество контрактов на одной странице:', resp_data['perpage'])

Всего контрактов: 1556
Количество контрактов на одной странице: 50


In [8]:
# Количество страниц должно быть целым числом
if resp_data['total'] % resp_data['perpage'] == 0:
    pages_number = resp_data['total'] // resp_data['perpage']

# Если количество страниц -- нецелое число, 
# то прибавим к результату целочисленного деления 1, 
# чтобы включить в выборку последнюю, неполную страницу 
else:
    pages_number = resp_data['total'] // resp_data['perpage'] + 1
    
print('Количество страниц для парсинга:', pages_number)

Количество страниц для парсинга: 32


## Структура data и нужные поля

Изучим структуру записи данных, чтобы определить, значение каких ключей внутри `data` необходимо получить для дальнейшего анализа. Чтобы не перегружать текущий блокнот, сохраним данные из первой записи в файл `one_record.json` и будем визуально ориентироваться на неё. 

In [9]:
first_record = resp_data['data'][0]

with open('data/one_record.json', 'w') as one_record_file:
    json.dump(obj=first_record, fp=one_record_file, indent=2, ensure_ascii=False)

При последующем анализе и парсинге выяснилось, что записи в ключе `contractProcedure` (информация о том, как подписывался конракт и его содержание) не универсальна -- большое количество контрактов содержит другую структуру. Поэтому содержимое ключа не является предметом интереса при сборе данных. 

`scan` -- ссылки на сканы документов, связанных с этим контрактом, `number` -- номер контракта. 

`currentContractStage` -- текущее состояние контракта, которое обозначается кодировкой в виде одной или двух английских заглавных букв, а именно: 

| Код	| Значение| 
|---|---|
| E	| Исполнение| 
| EC	| Исполнение завершено| 
| ET	| Исполнение прекращено| 
| IN	| Аннулирован| 

Проследим путь, который необходимо пройти по ключам, чтобы получить эту информацию из записи.

In [10]:
first_record['currentContractStage']

'E'

`id` -- некий идентификационный код записи или контракта с неопределённым назначением; `fileVersion` -- версия файла, также с неопредёленным форматом записи. 

`regionCode` -- код региона заказчика. Проследим путь к этому ключу -- то, каким образом можно извлечь информацию о коде региона заказчика. 

In [11]:
first_record['regionCode']

'77'

`contractUrl` -- ссылка на страницу контракта на официальном сайте государственных закупок ([zakupki.gov.ru](zakupki.gov.ru)). Сохраним эту ссылку в набор данных, чтобы иметь возможность быстро обращаться к информации о контракте -- для этого проследим путь к значению этого ключа. 

In [12]:
first_record['contractUrl']

'http://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber=1770420620122000099'

`singleCustomerReason` -- ключ, внутри которого содержится словарь с обоснованием единственного поставщика услуг (сами услуги описываются дальше). 

`signDate` -- дата подписания контракта. Проследим путь к этому значению. 

In [13]:
first_record['signDate']

'2022-12-29T00:00:00'

`price` -- цена за поставляемые услуги или товары, заявленная в контракте (может отличаться от реальной стоимости услуг или товаров, которые на самом деле были поставлены). Проследим путь к этому значению. 

In [14]:
first_record['price']

3815000.0

`documentBase` -- федеральный закон или иной нормативный акт, согласно с которым осуществляется государственная закупка; `foundation` -- словарь, который содержит внутри себя информацию, скорее всего, о том, что связано с источниками финансирования. 

`fz` -- номер федерального закона, по нормативам которого осуществляется федеральная закупка. Федеральный закон определяется долей государственной собственности в организации и целью закупки. 

Предположительно, в закупках этой организации будет использоваться 44-ФЗ, так как государственная доля в МИД -- 100% (это полностью государственное предприятие). 

In [15]:
first_record['fz']

'44'

`publishDate` -- дата публикации информации о контракте на сайте государственных закупок. По 44-ФЗ у заказчика есть пять рабочих дней с момента подписания контракта на то, чтобы отправить данные в ЕИС; при расторжении контракта по решению суда -- также пять рабочих дней; при расторжении в одностороннем порядке -- три рабочих дня. 

`regNum` -- идентификационный код закупки, который указывается в карточке проекта. Проследим путь к значению этого ключа: по номеру контракта всегда можно будет найти информацию о нём. 

In [16]:
first_record['regNum']

'1770420620122000099'

`versionNumber` -- неопределённый номер версии; `schemaVersion` -- неопределённая схема взаимодействия кого-то с кем-то. 

`execution` -- информация о датах, когда осуществлось действие контракта. В этом словаре существует два варианта структуры:

либо: 
* `startDate` -- дата начала действия контракта;
* `endDate` -- дата конца действия контракта. 

либо: 
* `month` -- месяц конца действия контракта;
* `year` -- год конца действия контракта. 

In [17]:
first_record['execution']['startDate']

'2023-01-01'

In [18]:
first_record['execution']['endDate']

'2024-03-31'

`customer` -- словарь, в котором содержится информация по заказчику. Так как данные собираются только по одной организации-заказчику, сохранять информацию из этого словаря нет смысла. 

`currency` -- словарь с информацией о валюте, в которой заключён контракт. Независимо от валюты, все цифры, связанные с деньгами, представлены в рублях. Обозначим путь к коду валюты. 

In [19]:
first_record['currency']['code']

'RUB'

`suppliers` -- список словарей, где каждый словарь -- это информация об одном поставщике. Определим путь к следующим данным внутри каждой записи о поставщике: название организации-поставщика, ИНН организации-поставщика, тип организации-поставщика.

Типы организации-поставщика: 

| Код | Значение| 
|---|---|
| U | Юридическое лицо | 
| P | Физическое лицо (ИП) |

In [20]:
first_record['suppliers'][0]['organizationName']

'ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ПРЕЗИДЕНТ-СЕРВИС" УПРАВЛЕНИЯ ДЕЛАМИ ПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ'

In [21]:
first_record['suppliers'][0]['inn']

'7730050504'

In [22]:
first_record['suppliers'][0]['participantType']

'U'

`products` -- список словарей, где каждый словарь -- это информация об одном товаре или об одной услуге, закупленном(-ой) в рамках текущего контракта. 

* `OKEI` -- словарь с обозначением единицы измерения товара или услуги, которые закупаются в рамках контракта по текущему продукту (обозначения установлены в [классификаторе](https://www.consultant.ru/document/cons_doc_LAW_53447/3308dd3043911d8e4fc498c856fd3ba547580b3d/));
* `price` -- цена, установленная за единицу продукта. 
* `OKPD2` -- словарь с кодом и наименованием обозначения типа продукции (обозначения установлены в [классификаторе](okpd2.json));
* `sid` -- неопределённый индетификатор безопасности; 
* `sum` -- итоговое количество средств, потраченных на закупку товара (в идеале совпадает со значением цены, умноженной на количество закупленного продукта);
* `quantity` -- количество продукта. 

In [24]:
first_record['products'][0]['OKPD2']['code']

# В контрактах, заключённых до 2014 года 
# first_record['products'][0]['OKPD']['code']

'52.23.11.000'

In [25]:
first_record['products'][0]['price']

3815000.0

In [26]:
first_record['products'][0]['quantity']

'1'

In [27]:
first_record['products'][0]['sum']

3815000.0

`printFormUrl` -- информация о заключенном контракте (его изменении) в подготовленном для печати виде. 

In [28]:
first_record['printFormUrl']

'https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=78707054'

# Парсинг

Парсинг выполняется с помощью цикла for, внутри которого находится ещё несколько циклов итерации. Результаты сохраняются в словарь `contracts_tab`, который впоследствии будет преобразован в датафрейм. `contracts_tab` представляет собой словарь словарей, где каждый словарь -- это информация об одном продукте в контракте; если продуктов в контракте несколько, то каждый продукт написан на новой строке. 

Перерыв между запросами -- 2 секунды. 

In [29]:
contracts_tab = list()

for page in range(1, pages_number + 1):
    # Запрос страницы
    url = f'http://openapi.clearspending.ru/restapi/v3/contracts/select/?customerinn=7704206201&sort=-signDate&page={page}'
    response = requests.get(url)
    # Вывод статуса запроса 
    print(page, response.status_code)
    
    # Получение данных из результата запроса 
    resp_data = response.json()['contracts']
    # Итерация через строки в наборе данных 
    for rec in resp_data['data']:
        # Итерация через продукты в записи 
        # для разделения отдельных продуктов на отдельные строки
        for product in rec.get('products'):
            contract = list()
            
            # Идентификационный код контракта 
            regNum = rec.get('regNum')
            contract.append(regNum)
            # Статус действия контракта
            currentContractStage = rec.get('currentContractStage')
            contract.append(currentContractStage)

            # Дата подписания контракта
            signDate = rec.get('signDate')
            contract.append(signDate)

            # Срок действия контракта 
            execution = rec.get('execution', {})

            startDate = execution.get('startDate')
            contract.append(startDate)

            endDate = execution.get('endDate')
            contract.append(endDate)

            execution_month = execution.get('month')
            contract.append(execution_month)

            execution_year = execution.get('year')
            contract.append(execution_year)

            # Код региона заказчика 
            regionCode = rec.get('regionCode')
            contract.append(regionCode)

            # Номер ФЗ
            fz = rec.get('fz')
            contract.append(fz)

            # Код валюты, в которой выполняется контракт 
            currency_code = rec.get('currency', {}).get('code')
            contract.append(currency_code)

            # Общая цена контракта
            contract_price = rec.get('price')
            contract.append(contract_price)

            # Поставщики 
            supps = list()
            try: 
                suppliers = rec['suppliers']
                for sup in suppliers:
                    sup_organizationName = sup.get('organizationName')
                    sup_inn = sup.get('inn')
                    sup_participantType = sup.get('participantType')
                    
                    sup_list = [sup_organizationName, sup_inn, sup_participantType]
                    supps.append(sup_list)
            except: pass
            contract.append(supps)
                
            # Информация о продукте 
            prod_OKPD2_code = product.get('OKPD2', {}).get('code')
            contract.append(prod_OKPD2_code)
            prod_OKPD_code = product.get('OKPD', {}).get('code')
            contract.append(prod_OKPD_code)
            prod_price = product.get('price')
            contract.append(prod_price)
            prod_quantity = product.get('quantity')
            contract.append(prod_quantity)
            prod_sum = product.get('sum')
            contract.append(prod_sum)
            
            # Ссылка на контракт на госзакупках 
            contractUrl = rec.get('contractUrl')
            contract.append(contractUrl)
            
            # Ссылка на контракт на печатную версию контракта 
            printFormUrl = rec.get('printFormUrl')
            contract.append(printFormUrl)
            
        contracts_tab.append(contract)
    
    time.sleep(2)

1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 200
20 200
21 200
22 200
23 200
24 200
25 200
26 200
27 200
28 200
29 200
30 200
31 200
32 200


# Обработка и сохранение результатов парсинга

Запишем наполненный при парсинге список в датафрейм. Каждый список внутри списка `contracts_tab` -- это одна строка в новом датафрейме. 

In [30]:
df = pd.DataFrame(contracts_tab)

Проверим размерность датафрейма и убедимся, что скачаны все имеющиеся в базе государственных закупок контракты, где заказчиком выступает организация с указанным при запросе ИНН. 

In [31]:
print('Размерность датафрейма:', df.shape)
print('Есть все контракты:', df.shape[0] == 1556)  # число известно с тестового парсинга

Размерность датафрейма: (1556, 19)
Есть все контракты: True


Переименуем колонки в датафрейме, чтобы они были представлены не числами, а отражали суть тех данных, которые в них хранятся. Порядок колонок определялся порядком записи данных об одном контракте при парсинге, поэтому его легко восстановить. 

In [32]:
column_names = {
    0: 'regNum', 1: 'currentContractStage', 2: 'signDate', 3: 'startDate', 4: 'endDate', 5: 'executionMonth',
    6: 'executionYear', 7: 'regionCode', 8: 'fz', 9: 'currencyCode', 10: 'contractPrice', 11: 'suppliers', 
    12: 'OKPD2', 13: 'OKPD', 14: 'prodPrice', 15: 'prodQuantity', 16: 'prodSum', 17: 'contractUrl', 18: 'printFormUrl',
}

df = df.rename(columns=column_names)

Убедимся, что данные в датафрейме не дублируются. Так как в колонке `suppliers` хранятся списки, исключим её из списка колонок, по которым необходимо проверять дубликаты (иначе возникнет ошибка `TypeError: unhashable type: 'list'`).

In [33]:
columns_can_duplicate = list(df.columns)
columns_can_duplicate.remove('suppliers')

df[df.duplicated(subset=columns_can_duplicate)]

,regNum,currentContractStage,signDate,startDate,endDate,executionMonth,executionYear,regionCode,fz,currencyCode,contractPrice,suppliers,OKPD2,OKPD,prodPrice,prodQuantity,prodSum,contractUrl,printFormUrl


Проверим количество пропущенных значений в датафрейме. 

In [34]:
df.isna().sum()

regNum                     0
currentContractStage       0
signDate                   0
startDate                629
endDate                  629
executionMonth           927
executionYear            927
regionCode                 0
fz                         0
currencyCode               0
contractPrice              0
suppliers                  0
OKPD2                    838
OKPD                    1281
prodPrice                  0
prodQuantity              70
prodSum                   61
contractUrl                0
printFormUrl              17
dtype: int64

Пропущенные значения в колонках `startDate`, `endDate` и в колонках `executionMonth`, `executionYear` взаимозаменяемы: если в записи есть значение в одной паре колонок, то другая пара колонок будет пустой. В сумме количество непустых значений в обеих колонках -- 1556, то есть во всём датафрейме в каждой записи присутствует временная отметка о дате, когда контракт исполнялся. В колонках `OKPD2` и `OKPD` ситуация аналогичная: ОКПД первой версии использовался до 2014 года, после чего был введён новый классификатор -- ОКПД2. 

Количество пропущенных значений в колонке `prodQuantity` превышает количество пропущенных значений в колонке `prodSum`. Так как количество закупленных продуктов -- это сумма (общая стоимость) / стоимость одного продукта, некоторые значения можно досчитать и тем самым дополнить значения в колонке. 

Посмотрим на типы данных и формат записи этих данных в датафрейме (на примере пяти случайных строк). Менять типы данных на специальные (категориальные, даты) нужно будет уже непосредственно при анализе, так как при сохранении датафрейма в csv или json эти специальные типы будут преобразованы в object или str. Перед сохранением же достаточно очистить записи от лишних знаков. 

In [35]:
display(df.dtypes)
display(df.sample(5))

regNum                   object
currentContractStage     object
signDate                 object
startDate                object
endDate                  object
executionMonth           object
executionYear            object
regionCode               object
fz                       object
currencyCode             object
contractPrice           float64
suppliers                object
OKPD2                    object
OKPD                     object
prodPrice               float64
prodQuantity             object
prodSum                 float64
contractUrl              object
printFormUrl             object
dtype: object

,regNum,currentContractStage,signDate,startDate,endDate,executionMonth,executionYear,regionCode,fz,currencyCode,contractPrice,suppliers,OKPD2,OKPD,prodPrice,prodQuantity,prodSum,contractUrl,printFormUrl
1114,0173100002213000062,E,2013-04-16T00:00:00,None,None,12,2013,77,94,RUB,1909047.5,"[[Общество с ограниченной ответственностью ""Це...",None,None,1909047.50,1,1909047.50,http://zakupki.gov.ru/epz/contract/contractCar...,http://zakupki.gov.ru/pgz/printForm?type=CONTR...
343,1770420620120000016,ET,2020-04-23T00:00:00,2019-10-01,2022-03-31,None,None,77,44,RUB,6263730.0,"[[ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВ...",68.32.12.000,None,6263730.00,1,6263730.00,http://zakupki.gov.ru/epz/contract/contractCar...,https://zakupki.gov.ru/epz/contract/printForm/...
128,1770420620121000081,E,2021-11-29T00:00:00,2021-11-29,2022-12-31,None,None,77,44,RUB,83350000.0,[[ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПР...,51.10.13.000,None,83350000.00,1,83350000.00,http://zakupki.gov.ru/epz/contract/contractCar...,https://zakupki.gov.ru/epz/contract/printForm/...
1327,0173100002212000004,E,2012-01-15T00:00:00,None,None,3,2012,77,94,RUB,6864000.0,[[Федеральное государственное унитарное предпр...,None,None,6864000.00,1,6864000.00,http://zakupki.gov.ru/epz/contract/contractCar...,http://zakupki.gov.ru/pgz/printForm?type=CONTR...
59,1770420620122000039,EC,2022-08-08T00:00:00,2022-08-08,2022-12-31,None,None,77,44,RUB,9999213.2,"[[ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФИ...",None,None,599.87,1,599.87,http://zakupki.gov.ru/epz/contract/contractCar...,https://zakupki.gov.ru/epz/contract/printForm/...


Досчитаем недостающее количество продуктов. 

In [36]:
df.prodQuantity = df.prodQuantity.astype('float')
df.prodQuantity = df.prodQuantity.fillna(df.prodSum / df.prodPrice)

print('Количество оставшихся пропущенных значений в колонке prodQuantity:', df.prodQuantity.isna().sum())

Количество оставшихся пропущенных значений в колонке prodQuantity: 56


В колонках `signDate`, `startDate`, `endDate` уберём выражение "T00:00:00", которое является частью стандартной временной отметки в json, получаемого через API, но по сути не несёт в себе никакой информации. 

In [37]:
df.signDate = df.signDate.str.replace('T00:00:00', '')
df.startDate = df.startDate.str.replace('T00:00:00', '')
df.endDate = df.endDate.str.replace('T00:00:00', '')

В колонках `executionMonth` и `executionYear` содержатся месяц и год начала действия контракта. Аналогичная, но более детализированная информация содержится в колонке `startDate`. Поскольку по своему смыслу две колонки крайне близки, объединим их: в колонку `startDate` вместо пропущенных ячеек добавим значения из `executionMonth` и `executionYear` (число поставим 1). 

Такое решение, с одной стороны, делает набор данных более структурированным, с другой -- накладывает ограничение на дальнейший анализ: невозможно использовать конкретные даты начала при статистических расчётах. После дополнения колонки `startDate`, столбцы `executionMonth` и `executionYear` можно удалить за ненадобностью.

In [38]:
df.endDate = df.endDate.fillna(df.executionYear + '-' + df.executionMonth + '-01')
df.drop(columns=['executionMonth', 'executionYear'], inplace=True)

В колонке `suppliers` содержится список списков поставщиком по данному контракту, причём про каждого поставщика известны название организации, ИНН организации и тип организации (физическое или юридическое лицо). Это разнородные данные, которые необходимо распаковать в отдельные столбцы. 

Для начала создадим три отдельных списка: список названий организаций (длина должна совпасть с длиной датафрейма), список ИНН организаций (длина должна совпасть с длиной датафрейма), список типов организаций (длина должна совпасть с длиной датафрейма). Известно, что как минимум в части строк список поставщиков по сути представляет собой перечисление нескольких вариаций названия организации, притом что ИНН у всех названий совпадает. 

С помощью цикла for проитерируем все строки в колонке поставщиков, причем названия организаций сразу приводятся к единому виду (все в верхнем регистре, кавычки одинаковые). Если по сути список поставщиков представляет собой поставщика, названного несколькими разными способами, то в список ИНН добавляется список, содержащий в себе только один ИНН; аналогично с типом компании (они совпадают в таком случае так же, как и ИНН). 

In [39]:
sup_names_column, sup_innslist_column, sup_listtypes_column = list(), list(), list()

for x in df.suppliers:  
    names_list = [sup[0].upper().replace('«', '"').replace('»', '"').replace("'", '"') for sup in x]
    inns_list = [sup[1] for sup in x]
    types_list = [sup[2] for sup in x]

    if len(set(inns_list)) == 1:
        sup_name = list(set(names_list))
        sup_inn = [inns_list[0]]
        sup_type = [types_list[0]]
    else:
        sup_name = names_list    
        sup_inn = inns_list
        sup_type = types_list

    sup_names_column.append(sup_name)
    sup_innslist_column.append(sup_inn)
    sup_listtypes_column.append(sup_type)

Убедимся, что размерность получишвихся трёх списков совпадает с размерностью датафрейма (иначе не получится преобразовать их в колонки). 

In [40]:
len(sup_names_column) == df.shape[0], len(sup_innslist_column) == df.shape[0], len(sup_listtypes_column) == df.shape[0]

(True, True, True)

Посмотрим,
- существуют ли строки, в которых содержится несколько названий организации, 
- существуют ли строки, в которых содержится несколько ИНН (то есть действительно было несколько поставщиков), 
- существуют ли строки, в которых содержится несколько типов компаний (то есть действительно было несколько поставщиков).  

In [41]:
for sup in sup_names_column:
    if len(sup) > 1:
        print(sup)

['ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ГЛАВНЫЙ ЦЕНТР СПЕЦИАЛЬНОЙ СВЯЗИ"', 'ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕННОГО УНИТАРНОГО ПРЕДПРИЯТИЯ "ГЛАВНЫЙ ЦЕНТР СПЕЦИАЛЬНОЙ СВЯЗИ"-УПРАВЛЕНИЕ СПЕЦИАЛЬНОЙ СВЯЗИ ПО Г.МОСКВЕ И МОСКОВСКОЙ ОБЛАСТИ']
['ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ГЛАВНОЕ ПРОИЗВОДСТВЕННО-КОММЕРЧЕСКОЕ УПРАВЛЕНИЕ ПО ОБСЛУЖИВАНИЮ ДИПЛОМАТИЧЕСКОГО КОРПУСА ПРИ МИНИСТЕРСТВЕ ИНОСТРАННЫХ ДЕЛ РОССИЙСКОЙ ФЕДЕРАЦИИ"', 'ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ФИЛИАЛ "МЕДИНЦЕНТР" ГЛАВУПДК ПРИ МИД РОССИИ']
['ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ГЛАВНОЕ ПРОИЗВОДСТВЕННО-КОММЕРЧЕСКОЕ УПРАВЛЕНИЕ ПО ОБСЛУЖИВАНИЮ ДИПЛОМАТИЧЕСКОГО КОРПУСА ПРИ МИНИСТЕРСТВЕ ИНОСТРАННЫХ ДЕЛ РОССИЙСКОЙ ФЕДЕРАЦИИ"', 'ФИЛИАЛ "МЕДИНЦЕНТР" ГЛАВУПДК ПРИ МИД РОССИИ']
['АКЦИОНЕРНОЕ ОБЩЕСТВО "ПОЧТА РОССИИ"', 'УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ ПОЧТОВОЙ СВЯЗИ Г. МОСКВЫ']
['ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ГЛАВНОЕ ПРОИЗВОДСТВЕННО-КОММЕРЧЕСКОЕ УПРАВЛЕНИЕ ПО ОБСЛУЖИВАНИЮ ДИП

In [42]:
for sup in sup_innslist_column:
    if len(sup) > 1:
        print(sup)

In [43]:
for sup in sup_listtypes_column:
    if len(sup) > 1:
        print(sup)

Так как строк, содержащих несколько ИНН или несколько типов компаний не найдено, можно заключить, что во всех строках, в которых содержится несколько названий компаний, с точки зрения ФНС речь идёт об одной организации. Поэтому преобразуем список ИНН и список типов компаний таким образом, чтобы в ячейках содержались не списки с единственными строками внутри них, а просто строки -- ИНН или тип компании. 

In [44]:
sup_inns_column = [inn[0] for inn in sup_innslist_column]
sup_types_column = [types[0] for types in sup_listtypes_column]

Собранные данные можно записать в качестве колонок в имеющийся датафрейм. Колонку `suppliers`, содержащую список списков (теперь распакованный и распределённый по столбцам), можно удалить. 

In [45]:
df['sup_name'] = sup_names_column
df['sup_inn'] = sup_inns_column
df['sup_type'] = sup_types_column

df.drop(columns=['suppliers'], inplace=True)

Предварительную обработку данных можно считать практически завершённой. Единственное, что нужно сделать ещё, -- убедиться, что все колонки в наборе данных содержат разнообразные данные (иначе нет смысла сохранять колонку, так как в ней одно-единственное значение). 

In [46]:
column_names = ['regNum', 'currentContractStage', 'signDate', 'startDate', 'endDate', 'regionCode', 
                'fz', 'currencyCode', 'contractPrice', 'OKPD2', 'OKPD', 'prodPrice', 'prodQuantity', 
                'prodSum', 'contractUrl', 'printFormUrl', 'sup_inn', 'sup_type']

for col in column_names:
    unique_num = len(df[col].unique())
    
    # Если в колонке содержится одно уникальное значение, 
    # код выведет название этой колонки. 
    # В противном случае программа не выведет ничего. 
    if unique_num == 1:
        print(col)

В предыдущей ячейке ничего не вывелось -- это значит, что все колонки содержат минимум 2 уникальных значения (и это значит, что они могут быть ценны для анализа). Предварительная обработка данных завершена, можно сохранять полученные результаты. 

Извлечем ссылки на контракты в отдельный датафрейм; чтобы иметь возможность обращаться к нему по номеру контракта, также запишем в датафрейм значение из колонки `regNum`. Также создадим второй датафрейм -- по сути, очищенный и готовый к последующему анализу. 

In [47]:
contracts_urls = df[['regNum', 'contractUrl', 'printFormUrl']]
df_clear = df[['regNum', 'currentContractStage', 'signDate', 'startDate', 'endDate', 
               'regionCode', 'fz', 'currencyCode', 'contractPrice', 'sup_name', 'sup_inn', 
               'sup_type', 'OKPD2', 'OKPD', 'prodPrice', 'prodQuantity', 'prodSum']]

Сохраним оба датафрейма в двух форматах: в csv и в json (позволяет сохранить колонку `sup_name` в качестве реального итерируемого списка, а не объекта, похожего на список). 

In [48]:
contracts_urls.to_json('data/contr_urls.json', orient='records')
contracts_urls.to_csv('data/contr_urls.csv')

df_clear.to_json('data/contr_data.json', orient='records')
df_clear.to_csv('data/contr_data.csv')

Также сохраним описание структуры данных для каждого из наборов. 

In [49]:
urls_struc = contracts_urls.dtypes.to_frame()
urls_struc['column'] = urls_struc.index
urls_struc['dtype'] = urls_struc[0]
urls_struc['non-null'] = [contracts_urls[col].count() for col in contracts_urls.columns]
urls_struc['descr'] = ['Идентификационный номер контракта', 
                       'URL на контракт на сайте zakupki.gov.ru', 
                       'URL на печатную версию контракта на сайте zakupki.gov.ru']

urls_struc.index = range(urls_struc.shape[0])
urls_struc.drop(columns=[0], inplace=True)

urls_struc.to_csv('data/contr_urls_struc.csv')

In [50]:
df_struc = df_clear.dtypes.to_frame()
df_struc['column'] = df_struc.index
df_struc['dtype'] = df_struc[0]
df_struc['non-null'] = [df_clear[col].count() for col in df_clear.columns]
df_struc['descr'] = ['Идентификационный номер контракта', 'Код текущего статуса контракта', 
                     'Дата подписания контракта', 'Дата начала действия контракта', 
                     'Дата окончания действия контракта', 'Код региона заказчика', 'Номер федерального закона', 
                     'Код валюты контракта', 'Общая цена контракта', 'Название организаций-поставщиков', 
                     'ИНН организации-поставщика', 'Тип организации-поставщика', 'Код ОКПД2 (с 2014 г.)', 
                     'Код ОКПД (до 2014 г.)', 'Стоимость 1 ед. продукта', 'Количество ед. продукта', 
                     'Сумма закупки данного продукта (цена * количество)']

df_struc.index = range(df_struc.shape[0])
df_struc.drop(columns=[0], inplace=True)

df_struc.to_csv('data/contr_data_struc.csv')